In [1]:


from minerva.models.nets.unet import UNet
from dasf.datasets import Dataset, DatasetArray
from dasf.pipeline import Pipeline
from dasf.pipeline.executors import DaskPipelineExecutor
import lightning as L
from dasf.ml.dl import LightningTrainer

[1715632748.588426] [d78f44b0045f:265496:f]        vfs_fuse.c:281  UCX  ERROR inotify_add_watch(/tmp) failed: No space left on device


In [2]:
class LabeledDataset(Dataset):
    def __init__(self, original_path, label_path, chunks=(1, -1, -1)):
        self.original = DatasetArray(
            name="input", root=original_path, chunks=chunks
        )
        self.label = DatasetArray(name="label", root=label_path, chunks=chunks)

    def load(self):
        self.original.load()
        self.label.load()
        return self

    def _lazy_load_cpu(self):
        return self.load()

    def _load_cpu(self):
        return self.load()

    def _lazy_load_gpu(self):
        return self.load()

    def _load_gpu(self):
        return self.load()

    def __len__(self):
        return len(self.original)

    def __getitem__(self, idx):
        return self.original[idx], self.label[idx]

In [3]:
original_path = "/workspaces/dasf/data/original.npy"
label_path = "/workspaces/dasf/data/envelope.npy"

model = UNet()
dataset = LabeledDataset(original_path, label_path)
trainer = LightningTrainer(
    model=model,
    use_gpu=False,
    max_epochs=1,
    limit_train_batches=16,
    unsqueeze_dim=0,
    strategy="ddp_notebook",    # Only for jupyter notebook
    devices=1,                  # Only for jupyter notebook
)

executor = DaskPipelineExecutor(
    local=True, use_gpu=False
)
pipeline = Pipeline(
    name="pipeline",
    executor=executor,
    verbose=True,
)

pipeline.add(trainer.fit, train_data=dataset)

pipeline.run()

/usr/local/lib/python3.10/dist-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36811 instead
  warnings.warn(
2024-05-13 20:39:15,625 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-fhewj3sb', purging
2024-05-13 20:39:15,625 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-r1ilx7aq', purging


[2024-05-13 20:39:16+0000] INFO - Beginning pipeline run for 'pipeline'
[2024-05-13 20:39:16+0000] INFO - Task 'LabeledDataset.load': Starting task run...
[2024-05-13 20:39:16+0000] INFO - Task 'LabeledDataset.load': Finished task run
[2024-05-13 20:39:16+0000] INFO - Task 'LightningTrainer.fit': Starting task run...


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/1
----------------------------------------------------------------------------------------------------
distributed_backend=gloo
All distributed processes registered. Starting with 1 processes
----------------------------------------------------------------------------------------------------

Missing logger folder: /workspaces/dasf/dasf-core/examples/tutorials/lightning_logs

  | Name     | Type     | Params
--------------------------------------
0 | backbone | _UNet    | 31.0 M
1 | fc       | Identity | 0     
2 | loss_fn  | MSELoss  | 0     
--------------------------------------
31.0 M    Trainable params
0      

Training: |          | 0/? [00:00<?, ?it/s]